In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
import os
from groq import Groq

In [6]:
class RAG:
    def __init__(self, collection_name="aus_food_nutrition"):
        self.model = SentenceTransformer('all-mpnet-base-v2')
        self.chroma_client = chromadb.Client()
        self.collection = self.chroma_client.get_or_create_collection(collection_name)
        self.doc_ids = set()  # Track existing IDs for updates

    def add_or_update_documents(self, docs):
        """
        docs: list of (doc_id, doc_text) tuples.
        If doc_id exists, it will update the doc; otherwise, it adds it.
        """
        ids, texts = zip(*docs)
        embeddings = self.model.encode(texts)
        for i, doc_id in enumerate(ids):
            if doc_id in self.doc_ids:
                # Delete before updating (Chroma requires this for updating)
                self.collection.delete(ids=[doc_id])
            self.collection.add(
                documents=[texts[i]], 
                embeddings=[embeddings[i]], 
                ids=[doc_id]
            )
            self.doc_ids.add(doc_id)

    def retrieve(self, prompt, n_results=2):
        """
        Returns the most relevant document texts for the given prompt.
        """
        query_embedding = self.model.encode([prompt])[0]
        results = self.collection.query(
            query_embeddings=[query_embedding], 
            n_results=n_results
        )
        return results["documents"][0] if "documents" in results else []


In [12]:
rag = RAG()
rag.add_or_update_documents([
    ("1", "Vegemite is a popular Australian spread made from brewers' yeast extract."),
    ("2", "Kangaroo meat is a lean source of protein, low in fat."),
    ("3", "The Australian Dietary Guidelines recommend eating a variety of foods from the five food groups.")
])

# Retrieve by prompt
analyzed_health_condition = """
{
    "obesity_prediction": {
        "obesity_level": "Overweight_Level_II"
        "confidence": 10%
    },
    "diabetes_prediction": {
        "diabetes": true,
        "confidence": 90%
    }
"""

weekly_plan_format = """{
"suggestion": STRING
"weekly_plan": [
    {
        "week": 1,
        "target_calories_per_day": INT,
        "focus": STRING,
        "workouts": [ARRAY OF STRINGS],
        "meal_notes": STRING,
        "reminders": [ARRAY OF STRINGS]
    },
    ... (repeat for as many weeks as appropriate)
}]"""
prompt = f"""
You are a nutrition and fitness assistant.
Below is an analyzed health condition for a user, expressed in JSON: {analyzed_health_condition}
Your task: Based on the analyzed health condition and using the retrieved knowledge, generate a weekly plan strictly in this JSON format (replace INT and STRING placeholders): {weekly_plan_format} 
"""
relevant_texts = rag.retrieve(prompt, n_results=2)
prompt = prompt + f"\nBelow is relevant knowledge about Australian foods and dietary guidelines: {relevant_texts}"
print("Retrieved texts:", relevant_texts)

Retrieved texts: ['The Australian Dietary Guidelines recommend eating a variety of foods from the five food groups.', 'Kangaroo meat is a lean source of protein, low in fat.']


In [13]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Based on the analyzed health condition, I've generated a weekly plan to help the user manage their overweight level and diabetes. Here's the plan in the required JSON format:

```
{
    "suggestion": "Consult a doctor or a registered dietitian before starting any new diet or exercise plan, especially with a diabetes diagnosis."
    "weekly_plan": [
        {
            "week": 1,
            "target_calories_per_day": 1800,
            "focus": "Portion control and balanced meals",
            "workouts": ["Brisk walking", "Light swimming", "Bodyweight exercises"],
            "meal_notes": "Eat a variety of foods from the five food groups, including lean proteins like kangaroo meat, and whole grains, fruits, and vegetables.",
            "reminders": ["Drink at least 8 glasses of water per day", "Monitor blood sugar levels regularly", "Take medication as prescribed"]
        },
        {
            "week": 2,
            "target_calories_per_day": 1700,
            "focus": "Increas